In [5]:
%matplotlib widget 

import ipywidgets as widgets
from IPython.display import display
import Berthing
import Catalogue
from SCN import SCN
from MV import MV
from Pnuematic import Pnuematic
import numpy as np

display(widgets.HTML(f"""
<head><link rel="stylesheet" href="custom_templates\static\WGA_style.css"></head>
"""))

def CreateLabel(value):

    wget = widgets.Label(
        value=value
    )
    
    wget.add_class("typ_text")

    return wget 

def CreateComboBox(description, placeholder, options):
    
    wget_label = CreateLabel(description)
    
    wget_combo_box = widgets.Combobox(
    placeholder=placeholder,
    options=options,
    ensure_option=True,
    disabled=False
    )   

    wget_combo_box.add_class("typ_input")

    wget = widgets.VBox([wget_label, wget_combo_box])
    return wget

def CreateToggleButtons(description, options):
    
    wget = widgets.ToggleButtons(
        description=description,
        options=options,
        disabled = False,
        button_style = '',
        orientation='horizontal'
    )
    return wget

def CreateSelect(description, options, default_val=None, style=None):
    
    if style is None:
        style = {
            'description_width':'initial'
        }        
    
    wget_label = CreateLabel(description)
    
    wget_select = widgets.Select(
        options=options,
        value=default_val,
        disabled=False,
        style=style
    )
    wget_select.add_class("typ_input")
    wget = widgets.VBox([wget_label,wget_select])

    return wget

def CreateFloatText(description, default_val, style=None):

    if style is None:
        style = {
            'description_width':'initial'
        }    

    wget_label = CreateLabel(description)

    wget_text = widgets.FloatText(
        value=default_val,
        style=style
    )
    wget_text.add_class("typ_input")
    
    wget = widgets.VBox([wget_label,wget_text])

    return wget

def CreateIntText(description, default_val, style=None):

    if style is None:
        style = {
            'description_width':'initial'
        }    

    wget_label = CreateLabel(description)

    wget_text = widgets.IntText(
        value=default_val,
        style=style,
    )
    wget_text.add_class("typ_input")

    wget = widgets.VBox([wget_label,wget_text])

    return wget

def CreateButton(description, style=None):
    
    if style is None:
        style = {
            'description_width':'initial',
        }        

    wget = widgets.Button(
        description=description,
        disabled = False,
        style=style
    )

    wget.add_class("typ_button")

    return wget

def CreateSelectionSlider(description, options, default_val, style=None):

    if style is None:
        style = {
            'description_width':'initial',
            'handle_color':'#FAA61A'
        }

    wget_label = CreateLabel(description)

    wget_slider = widgets.SelectionSlider(
        options=options,
        value=default_val,
        style=style,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
    wget_slider.add_class("typ_input")
    
    wget = widgets.VBox([wget_label, wget_slider])

    return wget

def CreateDropdown(description, options, default_val, style=None):
     
    if style is None:
        style = {
            'description_width':'initial'
        }        
   
    wget_label = CreateLabel(description)

    wget_dropdown = widgets.Dropdown(
        options=options,
        value=default_val,
        disabled=False,
        style=style
    )
    wget_dropdown.add_class("typ_input")
    wget = widgets.VBox([wget_label,wget_dropdown])

    return wget

def CreateRadioButtons(description, options, default_val):

    wget_label = CreateLabel(description)

    wget_radiobuttons = widgets.RadioButtons(
        options=options,
        value=default_val,
        disabled=False
    )
    wget_radiobuttons.add_class("typ_input")
    wget = widgets.VBox([wget_label, wget_radiobuttons])

    return wget

def DisplayWidget(widgets:list):
    
    if widgets is list:
        for w in widgets:
            display(w)
        return
    else:
        display(widgets)



HTML(value='\n<head><link rel="stylesheet" href="custom_templates\\static\\WGA_style.css"></head>\n')

<h2><b><span style="color:#FAA61A">BERTHING ENERGY</span><b></h2>
<p>Calculate the berthing energy for a given berthing event</p>

In [6]:
berthing_point = {
    "1/6 point": 0.333,
    "1/5 point": 0.3,
    "1/4 point": 0.25,
    "1/3 point": 0.1667
}

energy_output = widgets.Output()
@energy_output.capture(clear_output=True, wait=True)
def BerthingEnergy(b):
    try:
        abnormal_factor = abnormal_factor_w.children[1].value
        berthing_point_val = berthing_point.get(berthing_point_w.children[1].value)*LBP_w.children[1].value
        normal_energy = Berthing.BerthingEnergy(
            berthing_velocity=velocity_w.children[1].value,
            berthing_angle=berthing_angle_w.children[1].value,
            velocity_angle=velocity_angle_w.children[1].value,
            displacement=displacement_w.children[1].value,
            beam=beam_w.children[1].value,
            LBP=LBP_w.children[1].value,
            draft=draft_w.children[1].value,
            UKC=UKC_w.children[1].value,
            berthing_point=berthing_point_val,
            #hull_radius=hull_radius_w.value,
            softness_coefficient=softness_coefficient_w.children[1].value,
            configuration_coefficient=configuration_coefficient_w.children[1].value,
            mass_calc=mass_calc_w.children[1].value,
            output=False
        )
        design_energy = normal_energy*abnormal_factor

        if mass_calc_w.children[1].value == "Vasco Costa" and (
            velocity_w.children[1].value < 0.08 or 
            UKC_w.children[1].value < 0.1*draft_w.children[1].value):
                display(widgets.Label(
                     description="Vasco Costa is valid for UKC>=0.1*Draft and Berthing Velocity > 0.08m/s",
                     label_style="warning"
                ))

        normal_label = CreateLabel(f"Normal energy: {normal_energy:.2f}kNm")
        design_label = CreateLabel(f"Abnormal energy: {design_energy:.2f}kNm")

        display(normal_label)
        display(design_label)


    except ZeroDivisionError:
        warning_w = widgets.Label(
            value="Design inputs are incomplete. Review chosen values.",
            font_weight='bold',
            text_color='red'
            )
        display(warning_w)

        
#Mass coefficient method
mass_calc_w = CreateDropdown('Mass coefficient method',['Vasco Costa','Shigeru','PIANC'],'Vasco Costa')

#Berthing event parameters
berthing_angle_w = CreateFloatText("Berthing angle [°]", 5.0)
velocity_w = CreateFloatText("Berthing velocity [m/s]", 0.20)
velocity_angle_w = CreateFloatText("Velocity angle [°]", 0.0)
berthing_point_w = CreateSelectionSlider("Berthing point",berthing_point.keys(), "1/4 point")

berthing_event_container = widgets.HBox([berthing_point_w, berthing_angle_w, velocity_w, velocity_angle_w])

#Vessel parameters
displacement_w = CreateIntText("Displacement [t]", 3500)
LBP_w = CreateFloatText("LBP [m]",81.7)
beam_w = CreateFloatText("Beam [m]",18.0)
draft_w = CreateFloatText("Draft [m]",7.0)
UKC_w = CreateFloatText("UKC [m]",4.42)
hull_radius_w = CreateFloatText("Hull radius [m]", 20.6)

vessel_container = widgets.VBox([
    widgets.HBox([displacement_w,LBP_w, beam_w]),
    widgets.HBox([draft_w, UKC_w, hull_radius_w])
    ])

#Berthing coefficients
softness_coefficient_w = CreateFloatText("Softness coefficient [Cs]", 1.0)
configuration_coefficient_w = CreateFloatText("Configuration coefficient [Cc]",1.0)
abnormal_factor_w = CreateFloatText("Abnormal Factor", 2.0)
coefficients_container = widgets.HBox([abnormal_factor_w,softness_coefficient_w,configuration_coefficient_w])

#Accordian
berthing_container = widgets.Accordion(children=[berthing_event_container, vessel_container, coefficients_container])

berthing_container.set_title(0, "Berthing event")
berthing_container.set_title(1, "Vessel particulars")
berthing_container.set_title(2, "Berthing coefficients")

#Calculate buttone
calc_energy_btn = CreateButton("Calculate")
calc_energy_btn.on_click(BerthingEnergy)

#AppLayout containers
center_box = widgets.VBox([mass_calc_w, berthing_container])
footer_box = widgets.HBox([calc_energy_btn])

energy_app = widgets.AppLayout(
    header=None,
    left_sidebar=None,    
    center=center_box,
    right_sidebar=None,
    footer=footer_box,
    width='80%',
    alignment='center'
)

DisplayWidget(energy_app)
display(energy_output)


AppLayout(children=(HBox(children=(Button(description='Calculate', style=ButtonStyle(), _dom_classes=('typ_but…

Output()

<h2><b><span style="color:#FAA61A">FENDER DESIGN</span><b></h2>

In [7]:
fender_types = ["SCN","MV","Pnuematic"]

# Universal parameters
berthing_energy_w = CreateFloatText("Berthing Energy",0)
manufacturing_tolerance_w = CreateFloatText("Manufacturing Tolerance [%]",10)
max_temp_w = CreateFloatText("Max Temperature [°C]",23.0)
min_temp_w = CreateFloatText("Min Temperature [°C]",23.0)
calc_btn = CreateButton("Calculate")
design_parameters = widgets.HBox([berthing_energy_w, manufacturing_tolerance_w, max_temp_w, min_temp_w])
#DisplayWidget(design_parameters)

# SCN
SCN_w = CreateSelect('Chose Fender:',Catalogue.SCN,'SCN300')
SCN_grade_w = CreateSelect('Chose Fender Grade:',Catalogue.SCN_grades,0.9)
SCN_material_w = CreateSelect('Chose fender material:', ["Blend", "Rubber", "Synthetic"],"Blend")

SCN_container = widgets.HBox([SCN_w,SCN_grade_w, SCN_material_w])

# MV
MV_w = CreateSelect('Chose Fender:',Catalogue.MV,"300x600")
MV_compound_w = CreateSelect('Chose Compound Type',['A','B'], 'A')
MV_unit_spacing_w = CreateIntText("Unit spacing [mm]", 0)
MV_bow_flare_w = CreateFloatText("Bow Flare angle [°]", 0.0)

MV_container = widgets.HBox([MV_w,MV_compound_w,widgets.VBox([MV_unit_spacing_w,MV_bow_flare_w])])

# Pnuematic
pnuematic_w = CreateSelect('Chose Fender', Catalogue.pnuematic, "500x1000")
pnuematic_pressure_w = CreateSelect('Chose inflation pressure', [50, 80], 50)

Pnuematic_container = widgets.HBox([pnuematic_w,pnuematic_pressure_w])

fender_tab = widgets.Tab(children=[SCN_container, MV_container, Pnuematic_container])
for i in range(len(fender_tab.children)):
    fender_tab.set_title(i, fender_types[i])

#DisplayWidget(fender_tab)

fender_output = widgets.Output()
@fender_output.capture(clear_output=True, wait=True)
def FenderChart(b):
    fender = FenderCalc()
    berthing_energy = berthing_energy_w.children[1].value
    berthing_angle = berthing_angle_w.children[1].value
    velocity = velocity_w.children[1].value
    max_temp = max_temp_w.children[1].value
    min_temp = min_temp_w.children[1].value

    design_fender = fender_types[fender_tab.selected_index]

    #TODO add back in capacity factor. It's in SCN section below
    if berthing_energy > fender.rated_energy:
        print("Rated capacity of the fender is exceeded. Select another size or grade.")
    elif design_fender == "MV":
        flare_angle = MV_bow_flare_w.children[1].value
        fender.fender_chart(berthing_angle, flare_angle, velocity, max_temp, min_temp, berthing_energy)
    elif design_fender == "Pnuematic":
        fender.fender_chart(berthing_energy)
    else:
        fender.fender_chart(berthing_angle, velocity, max_temp, min_temp, berthing_energy) 
    
def FenderCalc():

    fender_type = fender_types[fender_tab.selected_index]
    manufacturing_tolerance = manufacturing_tolerance_w.children[1].value/100

    if fender_type == 'SCN':
        size = SCN_w.children[1].value
        grade = SCN_grade_w.children[1].value
        material = SCN_material_w.children[1].value
        fender = SCN(size,
                     grade,
                     manufacturing_tolerance,
                     manufacturing_tolerance, 
                     material)
        #energy_factor, reaction_factor = fender.capacity_factor(berthing_angle, velocity, temp)
        return fender
        
    elif fender_type == 'MV':
        size = MV_w.children[1].value
        compound = MV_compound_w.children[1].value
        leg_spacing = MV_unit_spacing_w.children[1].value

        fender = MV(size,
                    compound,
                    leg_spacing,
                    manufacturing_tolerance,
                    manufacturing_tolerance)
        return fender
        
    elif fender_type == 'Pnuematic':
        size = pnuematic_w.children[1].value
        pressure = pnuematic_pressure_w.children[1].value
        fender = Pnuematic(size,
                           0,
                           manufacturing_tolerance,
                           pressure)
        return fender
        
calc_btn.on_click(FenderChart)
berthing_app = widgets.AppLayout(
    header=design_parameters,
    left_sidebar=None,    
    center=fender_tab,
    right_sidebar=None,
    footer=calc_btn,
    width='80%',
    alignment='center'
)
DisplayWidget(berthing_app)
display(fender_output)

AppLayout(children=(HBox(children=(VBox(children=(Label(value='Berthing Energy', _dom_classes=('typ_text',)), …

Output()

<h2><b><span style="color:#FAA61A">FENDER OPTIMISATION</span><b></h2>

In [8]:
selection_w = CreateSelect('',[])

def CriteriaChange(change):
    label = selection_w.children[0]
    selection = selection_w.children[1]
    
    design_criteria = design_criteria_w.children[1].value
    fender_type =fender_types[fender_tab.selected_index]
       
    if design_criteria == 'Depth':
        label.value = "Select fender:"
    else:
        label.value = "Select grade:"
    if fender_type == 'SCN':
        if design_criteria == 'Depth':
            selection.options = Catalogue.SCN
        else:
            selection.options = list(
                map(lambda orig_string: f"F{orig_string}",Catalogue.SCN_grades)
            )  
    elif fender_type == 'MV':
        if design_criteria == 'Depth':
            selection.options = Catalogue.MV
        else:    
            selection.options = ['A','B']
    elif fender_type == 'Pnuematic':
        if design_criteria == 'Depth':
            selection.options = Catalogue.pnuematic
        else:
            selection.options = [50, 80]

    selection.value = selection.options[0]
    


optimal_fender_w = widgets.Output()

def first_index(val, search_list):
    index = next(
        (index for index, item in enumerate(search_list) if item > val), None
    )
    return index

@optimal_fender_w.capture(clear_output=True, wait=True)
def OptimalFender(b):
    berthing_energy = berthing_energy_w.children[1].value
    design_criteria = design_criteria_w.children[1].value
    selection = selection_w.children[1].value
    fender_type = fender_types[fender_tab.selected_index]    

    berthing_angle = berthing_angle_w.children[1].value
    velocity = velocity_w.children[1].value
    max_temp = max_temp_w.children[1].value
    
    fender = FenderCalc()
    
    if fender_type == 'SCN':
        energy_capacities = np.array(Catalogue.SCN_ratings[1])

        angular_factor = fender.angle_factor(berthing_angle)[0]
        velocity_factor = fender.velocity_factor(velocity)
        temperature_factor = fender.temperature_factor(max_temp)

        reduction_factor = angular_factor*velocity_factor*temperature_factor

        if design_criteria == 'Depth':
            row = Catalogue.SCN.index(selection)
            design_energies = energy_capacities[row,:]*reduction_factor
            col = first_index(berthing_energy, design_energies)
            if col is not None: 
                print(f"{selection}-F{Catalogue.SCN_grades[col]}")
            else:
                print("This size is not sufficient, select larger fender.")
        else:
            col = Catalogue.SCN_grades.index(float(selection[1:]))
            design_energies = energy_capacities[:,col]*reduction_factor
            row = first_index(berthing_energy, design_energies)
            if row is not None:
                print(f"{Catalogue.SCN[row]}-{selection}")
            else: 
                print(" This grade is not sufficient, select higher grade.")

    elif fender_type == 'MV':
        if design_criteria == 'Depth':
            energy_capacities_A = Catalogue.MV_compound_A
            energy_capacities_B = Catalogue.MV_compound_B

        else:    
            iter_list = Catalogue.MV
    elif fender_type == 'Pnuematic':
        if design_criteria == 'Depth':
            iter_list = [50, 80]    
        else:
            iter_list = Catalogue.pnuematic

design_criteria_w = CreateRadioButtons('Design criteria', ['Depth','Grade'],'Depth')
design_criteria_w.children[1].observe(CriteriaChange, 'value')
fender_tab.observe(CriteriaChange,'value')
CriteriaChange(True)

fender_calc_btn = CreateButton("Calculate")
fender_calc_btn.on_click(OptimalFender)

fender_app = widgets.AppLayout(
    header=design_criteria_w,
    left_sidebar=None,    
    center=selection_w,
    right_sidebar=None,
    footer=fender_calc_btn,
    width='80%',
    alignment='center'
)

DisplayWidget(fender_app)
display(optimal_fender_w)



AppLayout(children=(VBox(children=(Label(value='Design criteria', _dom_classes=('typ_text',)), RadioButtons(_d…

Output()